# Random Forest Classifier – Dataset 1 (Heart Disease Prediction)

This notebook demonstrates how to **load a pre-trained Random Forest model** (previously trained on Dataset 1) and use it to make predictions for new patients. Additionally, a simple **Gradio interface** is provided for interactive input and real-time predictions.  

## Workflow Description

The workflow in this notebook follows these steps:

1. **Load pre-trained model and scaler**  
   - The Random Forest model and the StandardScaler (fitted during training) are loaded from the `../models/random_forest_ds1/` directory.  

2. **Define the prediction function** (`predict_patient`)  
   - Takes patient data as input.  
   - Converts the input into a DataFrame with the same feature structure used in training.  
   - Applies the scaler transformation to match the training distribution.  
   - Uses the Random Forest model to make a prediction.  
   - Returns the result as text: *Heart Disease* or *No Heart Disease*.  

3. **Build the Gradio interface**  
   - Gradio provides a graphical interface where users can enter patient data.  
   - When the **Predict** button is clicked, Gradio calls the `predict_patient` function.  
   - The prediction result is displayed instantly in the interface.  

**In short:**  
**User Input → Prediction Function (scaling + model) → Gradio Output**  



In [1]:
import gradio as gr
import pandas as pd
import numpy as np 
import joblib

In [2]:
data = pd.read_csv('../data/preprocessed_rf/dataset_1_preprocessed.csv')
data

,age,gender,heart_rate,pressure_high,pressure_low,glucose,kcm,troponin,heart_disease
0,64,1,66,160,83,160.0,1.80,0.012,0
1,21,1,94,98,46,296.0,6.75,1.060,1
2,55,1,64,160,77,270.0,1.99,0.003,0
3,64,1,70,120,55,270.0,13.87,0.122,1
4,55,1,64,112,65,300.0,1.08,0.003,0
...,...,...,...,...,...,...,...,...,...
1311,44,1,94,122,67,204.0,1.63,0.006,0
1312,66,1,84,125,55,149.0,1.33,0.172,1
1313,45,1,85,168,104,96.0,1.24,4.250,1
1314,54,1,58,117,68,443.0,5.80,0.359,1


In [3]:
model = joblib.load("../models/random_forest_ds1/random_forest_ds1.pkl")
scaler = joblib.load("../models/random_forest_ds1/scaler_ds1.pkl")

In [4]:
def predict_patient(age, gender, heart_rate, pressure_high, pressure_low, glucose, kcm, troponin):
    patient_dict = {
        "age": age,
        "gender": gender,
        "heart_rate": heart_rate,
        "pressure_high": pressure_high,
        "pressure_low": pressure_low,
        "glucose": glucose,
        "kcm": kcm,
        "troponin": troponin
    }
    new_patient = pd.DataFrame([patient_dict])
    new_patient_scaled = scaler.transform(new_patient)
    prediction = model.predict(new_patient_scaled)
    return "Result: Heart Disease" if prediction[0] == 1 else "Result: No Heart Disease"

In [5]:
with gr.Blocks() as demo:
    gr.Markdown("## Heart Disease Prediction")

    with gr.Row():
        with gr.Column():
            age = gr.Number(label="Age")
            gender = gr.Number(label="Gender (0=Female, 1=Male)")
            heart_rate = gr.Number(label="Heart Rate")
            pressure_high = gr.Number(label="Pressure High")
        with gr.Column():
            pressure_low = gr.Number(label="Pressure Low")
            glucose = gr.Number(label="Glucose")
            kcm = gr.Number(label="KCM")
            troponin = gr.Number(label="Troponin")

    output = gr.Textbox(label="Prediction", lines=2)

    btn = gr.Button("Predict")
    btn.click(
        predict_patient,
        inputs=[age, gender, heart_rate, pressure_high, pressure_low, glucose, kcm, troponin],
        outputs=output
    )

demo.launch(inline=False, share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://5a1d222dc056484805.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Note on `share=True`:**  
Using `share=True` allows you to generate a public link that can be accessed online with the trained model.  
Sometimes, an antivirus program may block this process.  
If that happens, you need to allow the application through your antivirus software in order to successfully generate the public link.
